# 7.3 Softening function and damage function
Using the softening function with embedded fracture energy derive a damage function that can be correctly embedded in a finite element calculation. Then, the softening is not applied to a zero-thickness decohesion plane but to a zone of a finite length. 

To import the softening function derived in the notebook 7.2 run the notebook first.

In [1]:
%run 7_2_Softening_law_with_embedded_fracture_energy.ipynb

<IPython.core.display.Javascript object>

ValueError: The same variable should be used in all univariate expressions being plotted.

ValueError: The same variable should be used in all univariate expressions being plotted.

In [ ]:
f_w

## How to apply the softening function to a zone of length $L_s$?

If we wish to embed the softening behavior into a finite element simulation we encounter the problem that the deformation is not described by the crack opening $w$. The finite element discretization assumes by definition a smooth stran field $verepilon$ and there is no notion of discontinuity. 

As a consequence, we need to account for a length of the softening zone  $L_s$ which is actually equivalent to the size of the finite element.

In [ ]:
L_s, epsilon = sp.symbols('L_s, varepsilon')

The crack opening $w$ is simply substituted by
\begin{align}
 w = \varepsilon L_\mathrm{s}
\end{align}
to obtain the softening function in terms of strains

In [ ]:
f_epsilon = f_w.subs({'w' : epsilon * L_s})
f_epsilon

Note the important fact that the integral of the softening function over the strains scales the dissipated energy by the term $1/L_\mathrm{s}$. 

In [ ]:
sp.integrate(f_epsilon, epsilon)

As a consequence, by changing the size of the softening zone we also change the total amount of the energy dissipated!!! This feature of the softening function will be exploited in the implementation of the finite elements.

## How to convert a softening function to damage function?

The shape of the softening function describes the process of deterioration - starting from an undamaged state and ending in a fully damage state. Let us establish an equivalence between softening and damage evolution by requiring that they describe the same kind of stress decay.

Considering the softening zone of the length $L_s$ let us describe the damage within this zone using the state variable $omega$ and the elastic modulus $E_c$

In [ ]:
omega, E_c = sp.symbols('omega, E_c')

Then, the constitute law to be used in a finite element of the zone is given as

In [ ]:
sigma = (1 - omega) * E_c * epsilon

The stress decay described by the damage law and by the softening law should be the same. Let them set equal and solve for the damage variable $omega$. Using the sympy solver we obtain the algebraic solution as

In [ ]:
omega_Gf = sp.solve(sigma - f_epsilon, omega)[0]
omega_Gf

This new damage function is defined using material parameters with a clear mechanical interpretation. Such kind of material law is attractive because it makes it possible to design tests that focus on an isolated phenomenon, i.e. the determination of the material strength, E modulus or fracture energy.  

Let us now visually verify the shape of the damage function.
The set of parameters is assembled in a dictionary and then
they are all substituted into the damage function.

In [ ]:
data_f = dict(E_c = 28000, f_t = 3, L_s = 1, G_f = 0.01)
omega_Gf_epsilon = omega_Gf.subs(data_f)
omega_Gf_epsilon

The damage function is only valid in an inelastic regime. Therefore, we have to quantify the onset of inelasticity first as
\begin{align}
\varepsilon_0 = \frac{f_t}{E_c}
\end{align}

In [ ]:
epsilon_0 = (f_t / E_c).subs(data_f)
epsilon_0

Then the damage function can be plotted as

In [ ]:
splot(omega_Gf_epsilon, (epsilon,epsilon_0,epsilon_0*100))

The corresponding stress strain curve has then the form

In [ ]:
sigma = sp.Piecewise( (E_c * epsilon, epsilon < epsilon_0),
    (( 1 - omega_Gf ) * E_c * epsilon, epsilon >= epsilon_0))
sigma

In [ ]:
splot(sigma.subs(data_f), (epsilon,0,epsilon_0*100))

Note how the stress strain function scales with the change of the fracture energy and of the zone length.

Larger fracture energy makes the stress-strain response more ductile, while smaller makes it brittle.

On the other hand, larger size of the softening zone makes the softening behavior more brittle and smaller size of the zone makes it more ductile. Why?

In [ ]:
g_f = sp.integrate(sigma, epsilon)

In [ ]:
g_f